### Tidy Data Exercise 1:
    start date: friday, July 1st 2022

**Attendance Data**

    Read the data from the attendance table and calculate an attendance percentage for each student. One half day is worth 50% of a full day, and 10 tardies is equal to one absence.

    You should end up with something like this:

name

Billy    0.5250 

Jane     0.6875

John     0.9125

Sally    0.7625

Name: grade, dtype: float64


----

In [2]:
import pandas as pd
import numpy as np
import env
from env import user, password, host, get_connection

In [3]:
# creating the MySQL url to connect to the attendance dataset
url = get_connection(user, password, host, "tidy_data")

In [4]:
# creating the query and "attendace" dataframe

query = ''' 
SELECT *
FROM attendance
'''

attendance = pd.read_sql(query, url)

In [5]:
# checking the query pulled correctly
attendance.head()

,Unnamed: 0,2018-01-01,2018-01-02,2018-01-03,2018-01-04,2018-01-05,2018-01-06,2018-01-07,2018-01-08
0,Sally,P,T,T,H,P,A,T,T
1,Jane,A,P,T,T,T,T,A,T
2,Billy,A,T,A,A,H,T,P,T
3,John,P,T,H,P,P,T,P,P


In [6]:
attendance.shape

(4, 9)

**<u>initial obersations:</u>**

* let's rename the first column as "student_name"
* data columns appear to be coded for the following terms:

( ) == attendance value

    - "P" = present (1)
    - "T" = tardy (0.9)
    - "A" = absent (0)
    - "H" = half-day (0.5)


In [7]:
# columns i want to end up with:
# student_name
# attendance_percantage

attendance = attendance.rename(columns={attendance.columns[0]: "student_name"}) # renaming the first column 

In [8]:
attendance.head()

,student_name,2018-01-01,2018-01-02,2018-01-03,2018-01-04,2018-01-05,2018-01-06,2018-01-07,2018-01-08
0,Sally,P,T,T,H,P,A,T,T
1,Jane,A,P,T,T,T,T,A,T
2,Billy,A,T,A,A,H,T,P,T
3,John,P,T,H,P,P,T,P,P


In [9]:
# "reassigning" values / or reenconding 
# can use pandas "replace" and dictionary method to look a

attendance = attendance.replace({"P": 1, "T": 0.9, "A": 0, "H": 0.5})
attendance.head() # checks out!

,student_name,2018-01-01,2018-01-02,2018-01-03,2018-01-04,2018-01-05,2018-01-06,2018-01-07,2018-01-08
0,Sally,1,0.9,0.9,0.5,1.0,0.0,0.9,0.9
1,Jane,0,1.0,0.9,0.9,0.9,0.9,0.0,0.9
2,Billy,0,0.9,0.0,0.0,0.5,0.9,1.0,0.9
3,John,1,0.9,0.5,1.0,1.0,0.9,1.0,1.0


In [10]:
# i am ready to tidy the data:
# will use panads "melt" method to convert the attendance dataset to long-form

attendance_long = attendance.melt(
    id_vars = "student_name", \
    var_name = "date", \
    value_name = "attendance"
)


In [11]:
attendance_long.head() # long-form checks out!

,student_name,date,attendance
0,Sally,2018-01-01,1.0
1,Jane,2018-01-01,0.0
2,Billy,2018-01-01,0.0
3,John,2018-01-01,1.0
4,Sally,2018-01-02,0.9


In [12]:
print(f"Initial table shape: {attendance.shape}")
print(f"Long-form table shape: {attendance_long.shape}")

Initial table shape: (4, 9)
Long-form table shape: (32, 3)


In [13]:
# now, i want to calculate the average of ea. student's attendance percentage
# i can do this by using a "groupby" method and getting the mean of each student's attendance observation

avg_attendance = attendance_long.groupby("student_name").attendance.mean()
avg_attendance # checks out!

student_name
Billy    0.5250
Jane     0.6875
John     0.9125
Sally    0.7625
Name: attendance, dtype: float64

----
### Tidy Data Exercise 2:

Coffee Levels

* Read the coffee_levels table
* Transform the data so that each carafe is in it's own column
* Is this the best shape for the data?

In [14]:
# getting the "coffee_levels" table from MySQL

query = ''' 
SELECT *
FROM coffee_levels
'''

coffee = pd.read_sql(query, url)

In [15]:
# check to verify that table loaded correctly
coffee.head() # checks out!

,hour,coffee_carafe,coffee_amount
0,8,x,0.816164
1,9,x,0.451018
2,10,x,0.843279
3,11,x,0.335533
4,12,x,0.898291


In [16]:
coffee

,hour,coffee_carafe,coffee_amount
0,8,x,0.816164
1,9,x,0.451018
2,10,x,0.843279
3,11,x,0.335533
4,12,x,0.898291
5,13,x,0.310711
6,14,x,0.507288
7,15,x,0.215043
8,16,x,0.183891
9,17,x,0.391560


In [17]:
coffee.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30 entries, 0 to 29
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   hour           30 non-null     int64  
 1   coffee_carafe  30 non-null     object 
 2   coffee_amount  30 non-null     float64
dtypes: float64(1), int64(1), object(1)
memory usage: 848.0+ bytes


In [18]:
shape01 = coffee.shape # let's check the initial shape
shape01

(30, 3)

In [19]:
# Transform the data so that each carafe is in it's own column.
# this will require a pivot_table (long form -> wide form tidy data)

coffee_wide = coffee.pivot_table(index="hour", columns="coffee_carafe", values="coffee_amount").reset_index()

In [20]:
coffee_wide

# i want to rename the folling:
# "coffe_carafe": "" empty string text (no name needed)
# ["x, y, z" columns] to carafe 01, coffee A, coffee B, coffee, C, etc. or more practically...
# "light_roast", "medium_roast", or "bold_roast"

coffee_carafe,hour,x,y,z
0,8,0.816164,0.189297,0.999264
1,9,0.451018,0.521502,0.915990
2,10,0.843279,0.023163,0.144928
3,11,0.335533,0.235529,0.311495
4,12,0.898291,0.017009,0.771947
5,13,0.310711,0.997464,0.398520
6,14,0.507288,0.058361,0.864464
7,15,0.215043,0.144644,0.436364
8,16,0.183891,0.544676,0.280621
9,17,0.391560,0.594126,0.436677


In [21]:
coffee_wide = coffee_wide.rename(columns={"x": "lt_roast_cup", "y": "md_road_cup", "z": "bld_roast_cup"})

In [22]:
coffee_wide.head()

coffee_carafe,hour,lt_roast_cup,md_road_cup,bld_roast_cup
0,8,0.816164,0.189297,0.999264
1,9,0.451018,0.521502,0.915990
2,10,0.843279,0.023163,0.144928
3,11,0.335533,0.235529,0.311495
4,12,0.898291,0.017009,0.771947


In [23]:
type(coffee_wide)

pandas.core.frame.DataFrame

In [24]:
coffee_wide = coffee_wide.rename_axis("", axis = 1) # deleting current the current index "column name" -- "coffee_carafe"
coffee_wide # checks out!

,hour,lt_roast_cup,md_road_cup,bld_roast_cup
0,8,0.816164,0.189297,0.999264
1,9,0.451018,0.521502,0.915990
2,10,0.843279,0.023163,0.144928
3,11,0.335533,0.235529,0.311495
4,12,0.898291,0.017009,0.771947
5,13,0.310711,0.997464,0.398520
6,14,0.507288,0.058361,0.864464
7,15,0.215043,0.144644,0.436364
8,16,0.183891,0.544676,0.280621
9,17,0.391560,0.594126,0.436677


In [25]:
# let's check the shape of the new table

print(f"Initial table shape: {shape01}")
print(f"Wide table shape: {coffee_wide.shape}")

Initial table shape: (30, 3)
Wide table shape: (10, 4)


In [26]:
# let's reduce the number of value decimal places
coffee_wide = coffee_wide.round(2)
coffee_wide.head()

,hour,lt_roast_cup,md_road_cup,bld_roast_cup
0,8,0.82,0.19,1.00
1,9,0.45,0.52,0.92
2,10,0.84,0.02,0.14
3,11,0.34,0.24,0.31
4,12,0.90,0.02,0.77


#### Conclusion:

    Is this the best shape for the data?

i believe the "wide-form" table is effective in this instance.

**Other supporting reasons include:**

* No Nulls in the dataset
* limited number of variables/features to look across
* since the "hour" column is a time element, it makes reading across rather than down more natural/easier
  

----
### Tidy Data Exercise 3:

**Cake Recipes**

Read the cake_recipes table. This data set contains cake tastiness scores for combinations of different recipes, oven rack positions, and oven temperatures.

* Tidy the data as necessary.
* Which recipe, on average, is the best? recipe b
* Which oven temperature, on average, produces the best results? 275
* Which combination of recipe, rack position, and temperature gives the best result? recipe b, bottom rack, 300 degrees

In [27]:
# let's pull the "cake_recipes" table from MySQL

query = ''' 
SELECT *
FROM cake_recipes
'''

cakes = pd.read_sql(query, url)

In [28]:
# checking the table
cakes

,recipe:position,225,250,275,300
0,a:bottom,61.738655,53.912627,74.414730,98.786784
1,a:top,51.709751,52.009735,68.576858,50.228470
2,b:bottom,57.095320,61.904369,61.196980,99.248541
3,b:top,82.455004,95.224151,98.594881,58.169349
4,c:bottom,96.470207,52.001358,92.893227,65.473084
5,c:top,71.306308,82.795477,92.098049,53.960273
6,d:bottom,52.799753,58.670419,51.747686,56.183110
7,d:top,96.873178,76.101363,59.571620,50.971626


In [29]:
# checking the shape
shape01 = cakes.shape
shape01

(8, 5)

In [30]:
# cheking the table's info:
cakes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8 entries, 0 to 7
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   recipe:position  8 non-null      object 
 1   225              8 non-null      float64
 2   250              8 non-null      float64
 3   275              8 non-null      float64
 4   300              8 non-null      float64
dtypes: float64(4), object(1)
memory usage: 448.0+ bytes


In [31]:
# the "recipe:position" column is contatinated with both the recipe used and the position of the cake piece itself, which appears to have been rated seperately
# let's split these two values into their own respective columns before "melting" the overall table down

cakes[["recipe", "tasted_area"]] = cakes["recipe:position"].str.split(":", expand=True)

In [32]:
col_lst = ["recipe", "tasted_area", "225", "250", "275", "300"]
cakes = cakes[col_lst]

In [33]:
cakes.head()

,recipe,tasted_area,225,250,275,300
0,a,bottom,61.738655,53.912627,74.414730,98.786784
1,a,top,51.709751,52.009735,68.576858,50.228470
2,b,bottom,57.095320,61.904369,61.196980,99.248541
3,b,top,82.455004,95.224151,98.594881,58.169349
4,c,bottom,96.470207,52.001358,92.893227,65.473084


In [34]:
cakes_long = pd.melt(cakes, \
    id_vars=["recipe", "tasted_area" ], \
        var_name="temperature", \
            value_name="score").round(1)

In [35]:
# checking the newly create "long-form" table
cakes_long.head()

,recipe,tasted_area,temperature,score
0,a,bottom,225,61.7
1,a,top,225,51.7
2,b,bottom,225,57.1
3,b,top,225,82.5
4,c,bottom,225,96.5


In [36]:
# comparing the shape to initial table:

print(f"Initial table shape: {shape01}")
print(f"Long table shape: {cakes_long.shape}")
print("-----------", "\n")
print(f"table shape difference: {cakes_long.shape[0] - shape01[0]} (number of rows), {cakes_long.shape[1] - shape01[1]} (number of cols)")

Initial table shape: (8, 5)
Long table shape: (32, 4)
----------- 

table shape difference: 24 (number of rows), -1 (number of cols)


-----
* Which recipe, on average, is the best? recipe b
* Which oven temperature, on average, produces the best results? 275
* Which combination of recipe, rack position, and temperature gives the best result? recipe b, bottom rack, 300 degrees

In [50]:
highest_avg_recipe = cakes_long.groupby(["recipe"]).score.mean().idxmax()
highest_avg_score = cakes_long.groupby(["recipe"]).score.mean().max()

In [54]:
print(f"The best average recipe is {highest_avg_recipe} with an average score of: {round(highest_avg_score, 2)}")

The best average recipe is b with an average score of: 76.74


In [39]:
cakes_long.head()

,recipe,tasted_area,temperature,score
0,a,bottom,225,61.7
1,a,top,225,51.7
2,b,bottom,225,57.1
3,b,top,225,82.5
4,c,bottom,225,96.5


In [56]:
best_temp = cakes_long.groupby("temperature").score.mean().idxmax()
best_temp_score =  cakes_long.groupby("temperature").score.mean().max()

In [58]:
print(f"The best temperature is {best_temp} degrees with an average score of: {round(best_temp_score, 2)}")

The best temperature is 275 degrees with an average score of: 74.89


In [59]:
best_recipe_position_and_temp = cakes_long.groupby(["recipe", "tasted_area", "temperature"]).score.mean().idxmax()
best_combo_score = cakes_long.groupby(["recipe", "tasted_area", "temperature"]).score.mean().max()

In [60]:
# recipe "b", "bottom" area, and at 300 degrees is the best rated out of all combinations in the dataset

In [62]:
print(f"The best cake combination is {best_recipe_position_and_temp} with an average score of: {round(best_combo_score, 2)}")

The best cake combination is ('b', 'bottom', '300') with an average score of: 99.2
